In [1]:
using AutomotiveDrivingModels
using AutoScenes
using Base.Test

INFO: Recompiling stale cache file /home/tim/.julia/lib/v0.6/AutoScenes.ji for module AutoScenes.


In [2]:
const FloatScene = Frame{Entity{Float64, Void, Int}};
function AutoScenes.Vars(scene::FloatScene, roadway::Void)
    x = scene[1].state
    return Vars(Float64[x], # vector containing the one variable
                [StateBounds(-x, 1-x)], # bounds are deltas so we can vary x by this much to remain in [0,1]
                [:x], # variable symbol
                [1])  # index of entity in the scene
end
function f1(
    vars::Vars,
    assignment::Assignment, # indeces of variables in vars
    roadway::Void,
    )::Float64

    return vars.values[assignment[1]] < 0.5
end
function f2(
    vars::Vars,
    assignment::Assignment, # indeces of variables in vars
    roadway::Void,
    )::Float64

    return vars.values[assignment[1]] ≥ 0.5
end;
function AutoScenes.assign_feature{F <: typeof(f1), Void}(
    f::F,
    scene::FloatScene,
    roadway::Void,
    vars::Vars,
    )

    return Assignment[(1,0)]
end
function AutoScenes.assign_feature{F <: typeof(f2), Void}(
    f::F,
    scene::FloatScene,
    roadway::Void,
    vars::Vars,
    )

    return Assignment[(1,0)]
end;

roadway = nothing
scenes = [
    FloatScene([Entity{Float64,Void,Int}(0.0,roadway,1)], 1),
    FloatScene([Entity{Float64,Void,Int}(1/4,roadway,1)], 1),
    FloatScene([Entity{Float64,Void,Int}(3/4,roadway,1)], 1),
    FloatScene([Entity{Float64,Void,Int}(3/4,roadway,1)], 1),
    FloatScene([Entity{Float64,Void,Int}(3/4,roadway,1)], 1),
    FloatScene([Entity{Float64,Void,Int}(1.0,roadway,1)], 1),
]
features = (f1, f2)
factorgraphs = [FactorGraph(features, scene, roadway) for scene in scenes];
model = FactorModel(features, [1.0,1.0]);

In [3]:
gen = FactorGraphSceneGenerator(model, Dict(:x => Normal(0.0,1.0)), 10)

FactorGraphSceneGenerator(burnin=10)

In [4]:
factorgraph = factorgraphs[1]
a = zeros(length(factorgraph.vars))
b = similar(a)
logPtilde_a = begin
    factorgraph.vars.values .+= a
    logPtilde_a = log_ptilde(gen.model.features, gen.model.weights, factorgraph.vars,
        factorgraph.assignments, factorgraph.roadway)
    factorgraph.vars.values .-= a
    return logPtilde_a
end

srand(0)
(a, logPtilde_a) = metropolis_hastings_step!(gen, factorgraph, a, b, logPtilde_a)

[0.679107]


([0.0], 1.0)

In [5]:
metropolis_hastings!(gen, factorgraph)

n_steps: 10
[0.586617]
[0.586617]
[0.651565]
[0.651565]
[0.137354]
[0.137354]
[0.534837]
[0.534837]
[0.188482]
[0.188482]
[0.408176]
[0.408176]
[0.319559]
[0.319559]
[0.430981]
[0.430981]
[0.904695]
[0.904695]
[0.142018]
[0.142018]


1-element Array{Float64,1}:
 0.142018